In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Example dataset with IT issues and descriptions
data = {
    'issue': [
        'printer error', 'network failure', 'authentication issue',
        'software crash', 'hardware failure'
    ],
    'description': [
        'Printer not responding', 'No internet access',
        'Cannot log in', 'Application crashes frequently',
        'Device hardware failure'
    ]
}

df = pd.DataFrame(data)

# Encode the issue labels into numerical values
label_encoder = LabelEncoder()
df['issue_encoded'] = label_encoder.fit_transform(df['issue'])

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['description'].tolist(), df['issue_encoded'].tolist(), test_size=0.2, random_state=42
)


In [2]:
from transformers import LlamaTokenizer, LlamaForSequenceClassification

# Load the correct LlamaTokenizer for OpenLLaMA
model_name = "openlm-research/open_llama_3b"
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Load the model (with sequence classification head)
num_classes = len(df['issue_encoded'].unique())  # Determine number of unique issue labels
model = LlamaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at openlm-research/open_llama_3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def tokenize_texts(texts):
    return tokenizer(texts, truncation=True, padding='max_length', max_length=512, return_tensors="pt")

# Tokenize the texts
train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.